In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install catboost -q

In [7]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

In [8]:
train_ = pd.read_csv( 'Train.csv')
test_ = pd.read_csv('Test.csv')
submission_ = pd.read_csv('SampleSubmission.csv')

In [9]:
%%time

from sklearn.model_selection import KFold
fold_train = []
kf = KFold(n_splits=5, shuffle=False)
for r, (train_index, test_index) in enumerate(kf.split(train_)):
  test = train_.iloc[test_index]

  X_test = []
  X_test_columns = test.columns
  for v in test.values:
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for i in index:
      for k in range(len(binary)):
        if k == i:
          binary_transformed = list(copy.copy(binary))
          binary_transformed[i] = 0
          X_test.append(list(info) + binary_transformed + [8+k])

  X_test = pd.DataFrame(X_test)
  X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
          'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
          '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
          'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_num']

  X_test['ID'] = [str(r)+'_'+str(i) for i in range(X_test.shape[0])]

  test = []
  test_columns = X_test.columns

  for v in X_test.values:
    info = v[:8]
    binary = v[8:-1]
    num = v[-1]
    index = [k for k, i in enumerate(binary) if i == 1]
    for k in range(len(binary)):
        if k not in index:
          if 8+k == num:
            test.append(list(info) + list(binary) + [test_columns[8+k]] + [1])
          else:
            test.append(list(info) + list(binary) + [test_columns[8+k]] + [0])
  test = pd.DataFrame(test)
  test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'target']

  test = test[['ID', 'product_pred', 'target']]
  test['ID X PCODE'] = test['ID'] + ' X ' + test['product_pred']
  del test['ID'], test['product_pred']
  test.columns = ['Label', 'ID X PCODE']

  path_0 = '/content/drive/My Drive/Zimnat/stack/'
  path_1 = '/content/drive/My Drive/Zimnat/STACK_LGBM_BASELINE_2/'
  
  path_2 = '/content/drive/My Drive/Zimnat/stack_LIGHTGBM_BASE_3/'
  
  path_4 = '/content/drive/My Drive/Zimnat/Stack_XGBOOST_RONNY/'
  path_5 = '/content/drive/My Drive/Zimnat/CNN_verification/'
  path_6 = '/content/drive/My Drive/Zimnat/FFN_verification/'
  path_7 = '/content/drive/My Drive/Zimnat/Stack_LGBM_RONNY/'
  

  submit1 = pd.read_csv(path_0 + '0_fold' + str(r) + '.csv')
  submit2 = pd.read_csv( path_1 +'1_fold' + str(r) + '.csv')
  submit3 = pd.read_csv(path_2 + '2_fold' + str(r) + '.csv')
  submit4 = pd.read_csv(path_4 + '0_fold' + str(r) + '.csv')
  submit5 = pd.read_csv(path_5 + '0_fold' + str(r) + '.csv')
  submit6 = pd.read_csv(path_6 + '0_fold' + str(r) + '.csv')
  submit7 = pd.read_csv(path_7 + '0_fold' + str(r) + '.csv')


  test = test.merge(submit1, how='left', on='ID X PCODE')
  test = test.merge(submit2, how='left', on='ID X PCODE')
  test = test.merge(submit3, how='left', on='ID X PCODE')
  test = test.merge(submit4, how='left', on='ID X PCODE')
  test = test.merge(submit5, how='left', on='ID X PCODE')
  test = test.merge(submit6, how='left', on='ID X PCODE')
  test = test.merge(submit7, how='left', on='ID X PCODE')



  test.columns = ['target', 'ID X PCODE', 'Label1', 
                  'Label2', 
                  'Label3','Label4','Label5','Label6','Label7']

  fold_train.append(test)

fold_train = pd.concat(fold_train)

CPU times: user 32.8 s, sys: 1.21 s, total: 34 s
Wall time: 1min 11s


In [10]:
%%time
test = []
test_columns = test_.columns

for v in test_.values:
  info = v[:8]
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  for k in range(len(binary)):
    if k not in index:
      test.append(list(info) + list(binary) + [test_columns[8+k]])
test = pd.DataFrame(test)
test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred',]

test = test[['ID', 'product_pred']]
test['ID X PCODE'] = test['ID'] + ' X ' + test['product_pred']
del test['ID'], test['product_pred']

path_0 = '/content/drive/My Drive/Zimnat/stack/'
path_1 = '/content/drive/My Drive/Zimnat/STACK_LGBM_BASELINE_2/'
path_2 = '/content/drive/My Drive/Zimnat/stack_LIGHTGBM_BASE_3/'

path_4 = '/content/drive/My Drive/Zimnat/Stack_XGBOOST_RONNY/'
path_5 = '/content/drive/My Drive/Zimnat/CNN_verification/'
path_6 = '/content/drive/My Drive/Zimnat/FFN_verification/'
path_7 = '/content/drive/My Drive/Zimnat/Stack_LGBM_RONNY/'


submit1 = pd.read_csv(path_0+ '0_main.csv') 
submit2 = pd.read_csv(path_1 + '1_main.csv')
submit3 = pd.read_csv(path_2+ '2_main.csv')
submit4 = pd.read_csv(path_4 + '0_main.csv')
submit5 = pd.read_csv(path_5 + '0_main.csv' )
submit6 = pd.read_csv(path_6 + '0_main.csv' )
submit7 = pd.read_csv(path_7 + '0_main.csv' )

test = test.merge(submit1, how='left', on='ID X PCODE')
test = test.merge(submit2, how='left', on='ID X PCODE')
test = test.merge(submit3, how='left', on='ID X PCODE')
test = test.merge(submit4, how='left', on='ID X PCODE')
test = test.merge(submit5, how='left', on='ID X PCODE')
test = test.merge(submit6, how='left', on='ID X PCODE')
test = test.merge(submit7, how='left', on='ID X PCODE')

test.columns = ['ID X PCODE', 'Label1',
                'Label2',
                'Label3','Label4','Label5','Label6','Label7']

CPU times: user 4.63 s, sys: 134 ms, total: 4.77 s
Wall time: 11.8 s


In [11]:
print(fold_train.shape , test.shape)

(1298055, 9) (197147, 8)


In [12]:
fold_train.head(10)

,target,ID X PCODE,Label1,Label2,Label3,Label4,Label5,Label6,Label7
0,0,0_0 X P5DA,2.062916e-06,0.000007,0.000104,1.736310e-08,1.666908e-16,1.147360e-09,2.877702e-08
1,0,0_0 X RIBP,3.998375e-05,0.000067,0.001975,2.170020e-05,1.672458e-11,5.417373e-09,5.373559e-05
2,0,0_0 X 8NN1,1.329303e-06,0.000004,0.000066,2.628379e-07,4.753043e-13,4.251525e-13,3.275651e-07
3,0,0_0 X 7POT,6.973029e-06,0.000003,0.000071,9.227806e-07,1.920112e-13,1.147974e-08,2.244441e-06
4,0,0_0 X 66FJ,7.274762e-06,0.000003,0.000082,1.527384e-06,7.657072e-13,1.536300e-10,1.889719e-06
5,0,0_0 X GYSR,1.562166e-07,0.000005,0.000173,2.205191e-08,3.675670e-18,4.052474e-17,1.215748e-09
6,0,0_0 X SOP4,4.194947e-06,0.000007,0.000069,7.010831e-07,1.309845e-11,8.906488e-10,1.546785e-06
7,1,0_0 X RVSZ,9.993137e-01,0.999873,0.975555,9.998621e-01,1.000000e+00,9.999809e-01,9.996944e-01
8,0,0_0 X PYUQ,6.207601e-05,0.000066,0.001077,1.067546e-05,7.839219e-11,5.281901e-07,2.282937e-05
9,0,0_0 X LJR9,1.166633e-05,0.000022,0.000423,5.775543e-07,2.612555e-10,5.869792e-08,2.424491e-06


In [13]:
test.head() 

,ID X PCODE,Label1,Label2,Label3,Label4,Label5,Label6,Label7
0,F86J5PC X P5DA,0.000002,0.000106,0.000351,4.981359e-07,0.000020,0.000009,0.000001
1,F86J5PC X RIBP,0.000395,0.000344,0.002852,5.070527e-04,0.000392,0.000355,0.000238
2,F86J5PC X 8NN1,0.000003,0.000065,0.000044,2.708316e-06,0.000002,0.000004,0.000004
3,F86J5PC X 7POT,0.000071,0.000061,0.000097,5.838094e-05,0.000015,0.000042,0.000044
4,F86J5PC X 66FJ,0.000073,0.000050,0.000036,1.086586e-04,0.000059,0.000060,0.000023


In [14]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

models = []
models.append(LGBMClassifier(random_state=17, num_trees=300, max_depth = 3))
models.append(LGBMClassifier(random_state=17, num_trees=500, max_depth = 5))
models.append(XGBClassifier(random_state=17, num_trees=300, max_depth = 3))
models.append(XGBClassifier(random_state=17, num_trees=500, max_depth = 5))
models.append(CatBoostClassifier(random_state=17, num_trees=500, max_depth=3))
models.append(CatBoostClassifier(random_state=17, num_trees=1000, max_depth=5))

In [15]:
%%time
for i in range(len(models)):
  models[i].fit(fold_train.drop(columns=['target', 'ID X PCODE']), fold_train['target'])

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Learning rate set to 0.415293
0:	learn: 0.1235919	total: 468ms	remaining: 3m 53s
1:	learn: 0.0491206	total: 825ms	remaining: 3m 25s
2:	learn: 0.0401953	total: 1.18s	remaining: 3m 15s
3:	learn: 0.0346775	total: 1.53s	remaining: 3m 9s
4:	learn: 0.0344611	total: 1.87s	remaining: 3m 5s
5:	learn: 0.0335233	total: 2.17s	remaining: 2m 58s
6:	learn: 0.0326606	total: 2.51s	remaining: 2m 56s
7:	learn: 0.0324866	total: 2.8s	remaining: 2m 52s
8:	learn: 0.0322433	total: 3.15s	remaining: 2m 51s
9:	learn: 0.0321258	total: 3.45s	remaining: 2m 49s
10:	learn: 0.0320646	total: 3.77s	remaining: 2m 47s
11:	learn: 0.0320257	total: 4.11s	remaining: 2m 46s
12:	learn: 0.0319850	total: 4.43s	remaining: 2m 45s
13:	learn: 0.0319188	total: 4.79s	remaining: 2m 46s
14:	learn: 0.0318856	total: 5.11s	remaining: 2m 45s
15:	learn: 0.0318648	total: 5.45s	remaining: 2m 44s
16:	learn: 0.0318329	total: 5.77s	remaining: 2m 43s
17:	learn: 0.0317876	total: 6.06s	remaining: 2m 42s
18:	learn: 0.0317463	total: 6.46s	remaining: 2m

In [16]:
predicts = []
for i in range(len(models)):
  predicts.append(models[i].predict_proba(test.drop(columns=['ID X PCODE']))[:, 1])

In [17]:
test['target'] = np.mean(predicts, axis=0)

In [18]:
true_values = []
for v in test_.values:
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  for k in test_.columns[8:][index]:
    true_values.append(v[0] + ' X ' + k)

In [19]:
test = test[['ID X PCODE', 'target']]
test.columns = ['ID X PCODE', 'Label']

In [20]:
for v in true_values:
  test = test.append({'ID X PCODE':v,'Label':1.0}, ignore_index=True)

In [21]:
test.reset_index(drop=True, inplace=True)

In [22]:
test.to_csv('Final_version_Binary_Stacking.csv', index=False)

In [23]:
from google.colab import files
files.download('Final_version_Binary_Stacking.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>